In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

import tensorflow as tf
from keras import backend as k


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

k.tensorflow_backend.set_session(tf.Session(config=config))

batch_size = 128
num_classes = 10
epochs = 30

Using TensorFlow backend.


In [2]:
def add_module(input):

    Conv2D_reduce = Conv2D(16, (1,1), strides=(2,2), activation='relu', padding='same')(input)
    Conv2D_1_1 = Conv2D(16, (1,1), activation='relu', padding='same')(input)
    Conv2D_3_3 = Conv2D(16, (3,3),strides=(2,2), activation='relu', padding='same')(Conv2D_1_1)
    Conv2D_5_5 = Conv2D(16, (5,5),strides=(2,2), activation='relu', padding='same')(Conv2D_1_1)
    MaxPool2D_3_3 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(input)
    Cov2D_Pool = Conv2D(16, (1,1), activation='relu', padding='same')(MaxPool2D_3_3)
    concat = Concatenate(axis=-1)([Conv2D_reduce,Conv2D_3_3,Conv2D_5_5,Cov2D_Pool])
    return concat

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 29s 0us/step


In [4]:

input = Input(shape=(img_height, img_width, channel,))

Conv2D_1 = Conv2D(64, (3,3), activation='relu', padding='same')(input)
MaxPool2D_1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(Conv2D_1)
BatchNorm_1 = BatchNormalization()(MaxPool2D_1)

Module_1 = add_module(BatchNorm_1)
Module_1 = add_module(Module_1)

Output = Flatten()(Module_1)
Output = Dense(num_classes, activation='softmax')(Output)

In [5]:
from keras import losses
model = Model(inputs=[input], outputs=[Output])

In [6]:

model.compile(loss=losses.categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [7]:

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 14s 287us/step - loss: 1.4572 - acc: 0.4803 - val_loss: 1.2746 - val_acc: 0.5462
Epoch 2/30
50000/50000 [==============================] - 10s 204us/step - loss: 1.1003 - acc: 0.6117 - val_loss: 1.0656 - val_acc: 0.6223
Epoch 3/30
50000/50000 [==============================] - 10s 198us/step - loss: 0.9623 - acc: 0.6617 - val_loss: 1.0603 - val_acc: 0.6331
Epoch 4/30
50000/50000 [==============================] - 10s 195us/step - loss: 0.8713 - acc: 0.6977 - val_loss: 1.1484 - val_acc: 0.6048
Epoch 5/30
50000/50000 [==============================] - 10s 198us/step - loss: 0.8113 - acc: 0.7169 - val_loss: 1.0323 - val_acc: 0.6601
Epoch 6/30
50000/50000 [==============================] - 10s 197us/step - loss: 0.7617 - acc: 0.7360 - val_loss: 1.1032 - val_acc: 0.6291
Epoch 7/30
50000/50000 [==============================] - 10s 195us/step - loss: 0.7237 - acc: 0.7488 - val_loss: 1.

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = score = model.evaluate(x_train, y_train, verbose=0)
print('Training loss:', score[0])
print('Training accuracy:', score[1])


Test loss: 1.2746585406303406
Test accuracy: 0.6849
Training loss: 0.3805230078148842
Training accuracy: 0.86228
